<a href="https://colab.research.google.com/github/2303A51758/MITS-Internship/blob/main/medium_level_task_3_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 3: Automate Form Data Processing & Storage Integration
✨ Features:
Receives form data (from a dictionary or simulated POST request)

Appends data to Google Sheet using the Google Sheets API

Includes error handling and logging

Supports rate-limit handling via exponential backoff



Step 1: Prerequisites
1. Enable Google Sheets API
Go to Google Cloud Console

Create a project → Enable Google Sheets API and Google Drive API

Create Service Account credentials and download the JSON key file

2. Share your target Google Sheet
Share the sheet with your service account's email (e.g. xyz@project.iam.gserviceaccount.com) with Editor access

Step 2: Install Required Libraries

In [1]:
pip install gspread google-auth


Step 3: Python Script – form_to_sheet.py

In [15]:
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
import logging
import time

# ========== Logging Setup ==========
logging.basicConfig(level=logging.INFO, filename="form_sync.log", filemode="a",
                    format="%(asctime)s - %(levelname)s - %(message)s")

# ========== Google Sheets Setup ==========
# Update the path to your service account JSON key file
# CHANGE THIS LINE to the actual path of your service account JSON key file
# Replace "/path/to/your/service_account.json" with the actual path to your downloaded JSON key file
SERVICE_ACCOUNT_FILE = "your/actual/path/to/your/service_account.json"  # <--- **UPDATE THIS LINE** with the correct path
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SHEET_NAME = "Internship Applications"

def connect_to_sheet():
    try:
        # Replace "path/to/your/service_account.json" with the actual path to your downloaded JSON key file
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        client = gspread.authorize(creds)
        sheet = client.open(SHEET_NAME).sheet1
        return sheet
    except Exception as e:
        # This exception is caught in the calling function (process_form_data)
        # and will be logged there as "Error connecting to Google Sheet".
        # Re-raising the exception to indicate a failure in connection.
        raise e # Re-raising the caught exception

# ========== Form Processing Logic ==========

def process_form_data(form_data):
    """Takes a dict of form data and appends it to Google Sheet"""
    sheet = None # Initialize sheet to None
    try:
        sheet = connect_to_sheet()
    except Exception as e:
        # The connection error is already logged in connect_to_sheet
        logging.error("Failed to connect to Google Sheet. Cannot process form data.")
        return # Exit the function if connection fails

    headers = ["Name", "Email", "Domain", "Submission Time"]
    try:
        # Check if the sheet is empty by getting all values.
        # If empty, append the header row.
        if not sheet.get_all_values():
            sheet.append_row(headers)

        row = [
            form_data["name"],
            form_data["email"],
            form_data["domain"],
            datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ]

        retries = 3
        for attempt in range(retries):
            try:
                sheet.append_row(row)
                logging.info(f"Form data written: {row}")
                break  # Exit retry loop on success
            except Exception as e:
                logging.warning(f"Attempt {attempt+1} to append row failed: {e}")
                time.sleep(2 ** attempt)  # Exponential backoff
        else:
            # This block executes if the loop completes without a break
            logging.error("Max retries reached. Failed to append row to Google Sheet.")

    except Exception as e:
        # This catch block is for errors occurring *after* a successful sheet connection
        logging.error(f"Failed to process form data after connecting to sheet: {e}")


# ========== Example Usage ==========
if __name__ == "__main__":
    test_form = {
        "name": "Alice Johnson",
        "email": "alice@example.com",
        "domain": "Data Science"
    }

    process_form_data(test_form)

ERROR:root:Failed to connect to Google Sheet. Cannot process form data.


Output Example in Google Sheets:
Name	Email	Domain	Submission Time
Alice Johnson	alice@example.com	Data Science	2025-06-01 14:35:22

✅ Optional: Hook with Web Form
If you're collecting form data via FastAPI, Flask, or Django, just call process_form_data(form.dict()) in your POST handler.

✅ Extras You Can Add
Email confirmation using Gmail API

Export to a database in parallel (e.g. PostgreSQL, MongoDB)

Google Forms response parser using Google Drive API

